# Libraries

In [55]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import nltk.data
import spacy

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ferencipeter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [56]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# Import Data

In [80]:
data = pd.read_csv('data/data_full.csv')

In [86]:
data.head()

,Candidate,Date,Headline,Site,Headline_Cleaned
0,trump,2016-10-01,Trump and the Intellectuals,New York Times,Trump and the Intellectuals
1,trump,2016-10-01,The Other Trump,New York Times,The Other Trump
2,trump,2016-10-01,Trump Infrastructure Plan’s Fatal Flaw,New York Times,Trump Infrastructure Plan’s Fatal Flaw
3,trump,2016-10-01,Donald Trump’s Pathetic Fraternity,New York Times,Donald Trump’s Pathetic Fraternity
4,trump,2016-10-01,How Could Anyone Vote for Trump?,New York Times,How Could Anyone Vote for Trump?


In [87]:
data.shape

(1573, 5)

In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1573 entries, 0 to 1572
Data columns (total 5 columns):
Candidate           1560 non-null object
Date                1560 non-null object
Headline            1560 non-null object
Site                1560 non-null object
Headline_Cleaned    1560 non-null object
dtypes: object(5)
memory usage: 61.5+ KB


# Text Cleaning

In [93]:
def text_cleanup(text):
    text = text.replace('RT ', '')
    text = text.replace('\n', '')
    clean_text = []
    for word in text.split(' '):
        if word.startswith('http://'):
            word = ''
        if word.startswith('https://'):
            word = ''
        if word.startswith('.@'):
            word = word[1:]
        if word.endswith('.'):
            word = word[:-1]
        clean_text.append(word)
    return ' '.join(clean_text)


for i in data.index.tolist():
    try:
        data.at[i,'Headline_Cleaned'] = text_cleanup(data['Headline'][i])
    except AttributeError:
        pass

In [95]:
#for i in data.index.tolist():
    #data.at[i,'Snippet_Cleaned'] = text_cleanup(data['Snippet'][i])

In [94]:
data.head()

,Candidate,Date,Headline,Site,Headline_Cleaned
0,trump,2016-10-01,Trump and the Intellectuals,New York Times,Trump and the Intellectuals
1,trump,2016-10-01,The Other Trump,New York Times,The Other Trump
2,trump,2016-10-01,Trump Infrastructure Plan’s Fatal Flaw,New York Times,Trump Infrastructure Plan’s Fatal Flaw
3,trump,2016-10-01,Donald Trump’s Pathetic Fraternity,New York Times,Donald Trump’s Pathetic Fraternity
4,trump,2016-10-01,How Could Anyone Vote for Trump?,New York Times,How Could Anyone Vote for Trump?


# Stop Words and Tokenization

In [96]:
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS

data['Headline_Keywords'] = ""
data['Headline_Stopwords'] = ""
data['Headline_Tokens'] = ""

for i in data.index.tolist():
    doc = nlp(data['Headline_Cleaned'][i])
    
    print('---------')
    print(doc)
    print('---------')
        
    token_list = []
    for token in doc:
        token_list.append(token.text)
    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 
    stopwords =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
        else:
            stopwords.append(word)
            
    data.at[i,'Headline_Keywords'] = ', '.join(filtered_sentence)
    data.at[i,'Headline_Stopwords'] = ', '.join(stopwords)    
    data.at[i,'Headline_Tokens'] = ', '.join(token_list)    
        
    print('---------')
    print(filtered_sentence)
    print(stopwords)
    print('---------')

---------
Trump and the Intellectuals
---------
---------
['Trump', 'Intellectuals']
['and', 'the']
---------
---------
The Other Trump
---------
---------
['Trump']
['The', 'Other']
---------
---------
Trump Infrastructure Plan’s Fatal Flaw
---------
---------
['Trump', 'Infrastructure', 'Plan', 'Fatal', 'Flaw']
['’s']
---------
---------
Donald Trump’s Pathetic Fraternity
---------
---------
['Donald', 'Trump', 'Pathetic', 'Fraternity']
['’s']
---------
---------
How Could Anyone Vote for Trump?
---------
---------
['Vote', 'Trump', '?']
['How', 'Could', 'Anyone', 'for']
---------
---------
Donald Trump Opens New Line of Attack on Hillary Clinton: Her Marriage
---------
---------
['Donald', 'Trump', 'Opens', 'New', 'Line', 'Attack', 'Hillary', 'Clinton', ':', 'Marriage']
['of', 'on', 'Her']
---------
---------
Report That Donald Trump Did Business in Cuba Ups the Ante in Florida
---------
---------
['Report', 'Donald', 'Trump', 'Business', 'Cuba', 'Ups', 'Ante', 'Florida']
['That', '

---------
The Sleaziness of Donald Trump
---------
---------
['Sleaziness', 'Donald', 'Trump']
['The', 'of']
---------
---------
Donald Trump’s Apology That Wasn’t
---------
---------
['Donald', 'Trump', 'Apology']
['’s', 'That', 'Was', 'n’t']
---------
---------
Donald Trump, Groper in Chief
---------
---------
['Donald', 'Trump', ',', 'Groper', 'Chief']
['in']
---------
---------
Tough Donald Trump Coverage, if a Little Late
---------
---------
['Tough', 'Donald', 'Trump', 'Coverage', ',', 'Little', 'Late']
['if', 'a']
---------
---------
Why Republicans Are Probably Stuck With Donald Trump
---------
---------
['Republicans', 'Probably', 'Stuck', 'Donald', 'Trump']
['Why', 'Are', 'With']
---------
---------
Donald Trump’s Lewd Comments About Women
---------
---------
['Donald', 'Trump', 'Lewd', 'Comments', 'Women']
['’s', 'About']
---------
---------
Lewd Donald Trump Tape Is a Breaking Point for Many in the G.O.P
---------
---------
['Lewd', 'Donald', 'Trump', 'Tape', 'Breaking', 'P

---------
Trump Has Called Climate Change a Chinese Hoax Beijing Says It Is Anything But
---------
---------
['Trump', 'Called', 'Climate', 'Change', 'Chinese', 'Hoax', 'Beijing', 'Says']
['Has', 'a', 'It', 'Is', 'Anything', 'But']
---------
---------
Enthusiasm for Trump in Poland Is Tempered by Distrust of Putin
---------
---------
['Enthusiasm', 'Trump', 'Poland', 'Tempered', 'Distrust', 'Putin']
['for', 'in', 'Is', 'by', 'of']
---------
---------
‘Never Trump’ Stalwarts Try to Focus on Policy, Not the Man
---------
---------
['‘', 'Trump', '’', 'Stalwarts', 'Try', 'Focus', 'Policy', ',', 'Man']
['Never', 'to', 'on', 'Not', 'the']
---------
---------
A Bleak Outlook for Trump’s Promises to Coal Miners
---------
---------
['Bleak', 'Outlook', 'Trump', 'Promises', 'Coal', 'Miners']
['A', 'for', '’s', 'to']
---------
---------
Trump Adviser Takes Stake in China Ride-Sharing Company
---------
---------
['Trump', 'Adviser', 'Takes', 'Stake', 'China', 'Ride', '-', 'Sharing', 'Company']
['

---------
Would Trump Have Won Trump University Cases? Evidence Says He Faced Hard Fight
---------
---------
['Trump', 'Won', 'Trump', 'University', 'Cases', '?', 'Evidence', 'Says', 'Faced', 'Hard', 'Fight']
['Would', 'Have', 'He']
---------
---------
In Scotland, Trump Built a Wall Then He Sent Residents the Bill
---------
---------
['Scotland', ',', 'Trump', 'Built', 'Wall', 'Sent', 'Residents', 'Bill']
['In', 'a', 'Then', 'He', 'the']
---------
---------
Trump Has Options for Undoing Obama’s Climate Legacy
---------
---------
['Trump', 'Options', 'Undoing', 'Obama', 'Climate', 'Legacy']
['Has', 'for', '’s']
---------
---------
Republicans Divided Between Romney and Giuliani for Secretary of State
---------
---------
['Republicans', 'Divided', 'Romney', 'Giuliani', 'Secretary', 'State']
['Between', 'and', 'for', 'of']
---------
---------
Donald Trump and the Lawsuit Presidency
---------
---------
['Donald', 'Trump', 'Lawsuit', 'Presidency']
['and', 'the']
---------
---------
How Tru

---------
Issues in Hillary Clinton’s Past Leave Her Muted in Furor Over Donald Trump
---------
---------
['Issues', 'Hillary', 'Clinton', 'Past', 'Leave', 'Muted', 'Furor', 'Donald', 'Trump']
['in', '’s', 'Her', 'in', 'Over']
---------
---------
As a Black Republican Senator, Tim Scott Faces a Unique Donald Trump Issue
---------
---------
['Black', 'Republican', 'Senator', ',', 'Tim', 'Scott', 'Faces', 'Unique', 'Donald', 'Trump', 'Issue']
['As', 'a', 'a']
---------
---------
Officials Fight Donald Trump’s Claims of a Rigged Vote
---------
---------
['Officials', 'Fight', 'Donald', 'Trump', 'Claims', 'Rigged', 'Vote']
['’s', 'of', 'a']
---------
---------
Donald Trump Says He’s a ‘Big Fan’ of Hindus
---------
---------
['Donald', 'Trump', 'Says', '‘', 'Big', 'Fan', '’', 'Hindus']
['He', '’s', 'a', 'of']
---------
---------
Peter Thiel to Donate $1.25 Million in Support of Donald Trump
---------
---------
['Peter', 'Thiel', 'Donate', '$', '1.25', 'Million', 'Support', 'Donald', 'Trump'

...

---------
Rep Devin Nunes: Trump transition is an 'organized process'
---------
---------
['Rep', 'Devin', 'Nunes', ':', 'Trump', 'transition', "'", 'organized', 'process', "'"]
['is', 'an']
---------
---------
Critics call for Trump Cabinet diversity
---------
---------
['Critics', 'Trump', 'Cabinet', 'diversity']
['call', 'for']
---------
---------
Trump transition team kicks into high gear
---------
---------
['Trump', 'transition', 'team', 'kicks', 'high', 'gear']
['into']
---------
---------
Flynn: Outspoken general, intelligence pro, Trump supporter
---------
---------
['Flynn', ':', 'Outspoken', 'general', ',', 'intelligence', 'pro', ',', 'Trump', 'supporter']
[]
---------
---------
McCain to Trump: Waterboarding won't be reinstated
---------
---------
['McCain', 'Trump', ':', 'Waterboarding', 'wo', 'reinstated']
['to', "n't", 'be']
---------
---------
Sanctuary cities, Trump administration on collision course
---------
---------
['Sanctuary', 'cities', ',', 'Trump', 'administra

---------
The Latest: Trump says Clinton win would threaten religions
---------
---------
['Latest', ':', 'Trump', 'says', 'Clinton', 'win', 'threaten', 'religions']
['The', 'would']
---------
---------
Pages 40-44 of FBI probe into Clinton emails
---------
---------
['Pages', '40', '-', '44', 'FBI', 'probe', 'Clinton', 'emails']
['of', 'into']
---------
---------
FBI agreed to destroy laptops of two Clinton aides
---------
---------
['FBI', 'agreed', 'destroy', 'laptops', 'Clinton', 'aides']
['to', 'of', 'two']
---------
---------
Clinton sought Pentagon, State Department contracts for Chelsea's friend
---------
---------
['Clinton', 'sought', 'Pentagon', ',', 'State', 'Department', 'contracts', 'Chelsea', 'friend']
['for', "'s"]
---------
---------
Peek: Clinton is not authentic
---------
---------
['Peek', ':', 'Clinton', 'authentic']
['is', 'not']
---------
---------
Email 'boxes' in Clinton case AWOL
---------
---------
['Email', "'", 'boxes', "'", 'Clinton', 'case', 'AWOL']
['in'

---------
Former Secret Service agent reveals the real Hillary Clinton
---------
---------
['Secret', 'Service', 'agent', 'reveals', 'real', 'Hillary', 'Clinton']
['Former', 'the']
---------
---------
Cozying up to Clinton
---------
---------
['Cozying', 'Clinton']
['up', 'to']
---------
---------
Clinton ally reportedly aided FBI official's wife
---------
---------
['Clinton', 'ally', 'reportedly', 'aided', 'FBI', 'official', 'wife']
["'s"]
---------
---------
Clinton and Warren join forces amid daily WikiLeaks scandals
---------
---------
['Clinton', 'Warren', 'join', 'forces', 'amid', 'daily', 'WikiLeaks', 'scandals']
['and']
---------
---------
Top adviser on Clinton Wall Street speeches: 'It's pretty bad'
---------
---------
['adviser', 'Clinton', 'Wall', 'Street', 'speeches', ':', "'", 'pretty', 'bad', "'"]
['Top', 'on', 'It', "'s"]
---------
---------
WikiLeaks emails blow hole in Obama story on Clinton server
---------
---------
['WikiLeaks', 'emails', 'blow', 'hole', 'Obama', 

---------
Democrats regroup as they grapple with Trump's victory
---------
---------
['Democrats', 'regroup', 'grapple', 'Trump', 'victory']
['as', 'they', 'with', "'s"]
---------
---------
Young mother bumps into Bill and Hillary Clinton on post-defeat walk
---------
---------
['Young', 'mother', 'bumps', 'Bill', 'Hillary', 'Clinton', 'post', '-', 'defeat', 'walk']
['into', 'and', 'on']
---------
---------
Trump adviser urges Obama, Clinton to speak out on protests
---------
---------
['Trump', 'adviser', 'urges', 'Obama', ',', 'Clinton', 'speak', 'protests']
['to', 'out', 'on']
---------
---------
Giuliani on Trump's agenda, potential pardon for Clinton
---------
---------
['Giuliani', 'Trump', 'agenda', ',', 'potential', 'pardon', 'Clinton']
['on', "'s", 'for']
---------
---------
Doug Schoen breaks down the Clinton collapse
---------
---------
['Doug', 'Schoen', 'breaks', 'Clinton', 'collapse']
['down', 'the']
---------
---------
Clinton blames election defeat on FBI director's let

TypeError: object of type 'float' has no len()

In [65]:
data['Snippet_Keywords'] = ""
data['Snippet_Stopwords'] = ""
data['Snippet_Tokens'] = ""

for i in data.index.tolist():
    doc = nlp(data['Snippet_Cleaned'][i])
    
    print('---------')
    print(doc)
    print('---------')
        
    token_list = []
    for token in doc:
        token_list.append(token.text)
    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 
    stopwords =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
        else:
            stopwords.append(word)
            
    data.at[i,'Snippet_Keywords'] = ', '.join(filtered_sentence)
    data.at[i,'Snippet_Stopwords'] = ', '.join(stopwords)    
    data.at[i,'Snippet_Tokens'] = ', '.join(token_list)    
        
    print('---------')
    print(filtered_sentence)
    print(stopwords)
    print('---------')

KeyError: 'Snippet_Cleaned'

In [97]:
data.head()

,Candidate,Date,Headline,Site,Headline_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens
0,trump,2016-10-01,Trump and the Intellectuals,New York Times,Trump and the Intellectuals,"Trump, Intellectuals","and, the","Trump, and, the, Intellectuals"
1,trump,2016-10-01,The Other Trump,New York Times,The Other Trump,Trump,"The, Other","The, Other, Trump"
2,trump,2016-10-01,Trump Infrastructure Plan’s Fatal Flaw,New York Times,Trump Infrastructure Plan’s Fatal Flaw,"Trump, Infrastructure, Plan, Fatal, Flaw",’s,"Trump, Infrastructure, Plan, ’s, Fatal, Flaw"
3,trump,2016-10-01,Donald Trump’s Pathetic Fraternity,New York Times,Donald Trump’s Pathetic Fraternity,"Donald, Trump, Pathetic, Fraternity",’s,"Donald, Trump, ’s, Pathetic, Fraternity"
4,trump,2016-10-01,How Could Anyone Vote for Trump?,New York Times,How Could Anyone Vote for Trump?,"Vote, Trump, ?","How, Could, Anyone, for","How, Could, Anyone, Vote, for, Trump, ?"


# Sentiment

#### "VADER has been found to be quite successful when dealing with social media texts, NY Times editorials, movie reviews, and product reviews. "

With VADER no need for removing stop words as its built-in and punctuation is part of the analysis

In [98]:
import spacy
from spacy.tokens import Doc
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nlp = spacy.load('en_core_web_sm') 
sentiment_analyzer = SentimentIntensityAnalyzer()
def polarity_scores(doc):
    return sentiment_analyzer.polarity_scores(doc.text)
 
Doc.set_extension('polarity_scores', getter=polarity_scores,force=True)

data['Sentiment_Values'] = ""
data['Sentiment'] = ""

for i in data.index.tolist():
    doc = nlp(data['Headline'][i])
    
    print('---------')
    print(doc)
    print('---------')
                
    senti = []
    senti.append(doc._.polarity_scores)
        
    data.at[i,'Sentiment_Values'] = senti   
    data.at[i,'Sentiment'] = sorted(senti[0], key=senti[0].__getitem__)[-1]
    
    print('---------')
    print(senti)
    print('---------')

---------
Trump and the Intellectuals
---------
---------
[{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'compound': 0.3818}]
---------
---------
The Other Trump
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump Infrastructure Plan’s Fatal Flaw
---------
---------
[{'neg': 0.467, 'neu': 0.533, 'pos': 0.0, 'compound': -0.5423}]
---------
---------
Donald Trump’s Pathetic Fraternity
---------
---------
[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'compound': -0.5719}]
---------
---------
How Could Anyone Vote for Trump?
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Donald Trump Opens New Line of Attack on Hillary Clinton: Her Marriage
---------
---------
[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compound': -0.4767}]
---------
---------
Report That Donald Trump Did Business in Cuba Ups the Ante in Florida
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
-------

---------
Tough Donald Trump Coverage, if a Little Late
---------
---------
[{'neg': 0.2, 'neu': 0.8, 'pos': 0.0, 'compound': -0.128}]
---------
---------
Why Republicans Are Probably Stuck With Donald Trump
---------
---------
[{'neg': 0.222, 'neu': 0.778, 'pos': 0.0, 'compound': -0.25}]
---------
---------
Donald Trump’s Lewd Comments About Women
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Lewd Donald Trump Tape Is a Breaking Point for Many in the G.O.P.
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Those Who Don’t Understand Trump Are Doomed to Repeat Him
---------
---------
[{'neg': 0.318, 'neu': 0.682, 'pos': 0.0, 'compound': -0.6369}]
---------
---------
Billy Bush Says He’s Ashamed by Lewd Talk With Donald Trump
---------
---------
[{'neg': 0.237, 'neu': 0.763, 'pos': 0.0, 'compound': -0.4767}]
---------
---------
Transcript of Donald Trump’s Videotaped Apology
---------
-------

---------
Ex-General and Critic of Islam to Help Guide Trump in White House
---------
---------
[{'neg': 0.142, 'neu': 0.676, 'pos': 0.182, 'compound': 0.1531}]
---------
---------
Muslim Americans Speak of Escalating Worries
---------
---------
[{'neg': 0.359, 'neu': 0.641, 'pos': 0.0, 'compound': -0.4215}]
---------
---------
Why Democrats Now Need the F.B.I. Director, James Comey
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
It’s Not Just the White House. Change Is in the Air on Wall St., Too
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Oh, No! Donald Trump’s Calling
---------
---------
[{'neg': 0.384, 'neu': 0.616, 'pos': 0.0, 'compound': -0.3595}]
---------
---------
Donald Trump and the Bieber Doctrine
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Indian Business Partners Hope to Exploit Their Ties to Donald Trump
---------
-------

---------
Trump Claims, With No Evidence, That ‘Millions of People’ Voted Illegally
---------
---------
[{'neg': 0.18, 'neu': 0.82, 'pos': 0.0, 'compound': -0.296}]
---------
---------
Combative, Populist Steve Bannon Found His Man in Donald Trump
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Invoking Radicals of America’s Recent Past in a Politically Fraught Present
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
South Korean Leader Digs In Against Rising Calls for Impeachment
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Fidel Castro, Donald Trump, Cyber Monday: Your Weekend Briefing
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Fidel Castro, François Fillon, Narendra Modi: Your Monday Briefing
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------


---------
Was That a Presidential Debate or a Pitch for Trump TV?
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Donald Trump’s Contempt for Democracy
---------
---------
[{'neg': 0.487, 'neu': 0.513, 'pos': 0.0, 'compound': -0.5859}]
---------
---------
A ‘Taco Wall’ to Protest Trump
---------
---------
[{'neg': 0.333, 'neu': 0.667, 'pos': 0.0, 'compound': -0.25}]
---------
---------
Donald Trump vs. American Democracy
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Donald Trump Is Wrong on Mosul Attack, Military Experts Say
---------
---------
[{'neg': 0.437, 'neu': 0.563, 'pos': 0.0, 'compound': -0.7351}]
---------
---------
Donald Trump Won’t Say if He’ll Accept Result of Election
---------
---------
[{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'compound': 0.3818}]
---------
---------
Trump and the ‘Rigged’ Emmy Awards: A History of Snubs
---------
---------
[{'neg': 0.237, 'neu': 0.534, '

...

---------
Press pummels Trump over debate
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Judith Miller: Women are responding to Trump by voting early
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Sean Spicer: Trump offered American voters a detailed plan
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump booed, media pounce
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Media's 'crazy' Trump talk
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Press pummels Trump over debate
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump acknowledges 'Blacks for Trump' supporters
---------
---------
[{'neg': 0.0, 'neu': 0.633, 'pos': 0.367, 'compound': 0.4404}]
---------
---------
Trump: 'No interest' in T

---------
Sanctuary cities, Trump administration on collision course
---------
---------
[{'neg': 0.294, 'neu': 0.706, 'pos': 0.0, 'compound': -0.3612}]
---------
---------
Trump team: Mattis a strong candidate for Cabinet post
---------
---------
[{'neg': 0.0, 'neu': 0.68, 'pos': 0.32, 'compound': 0.5106}]
---------
---------
Fox News Reporting: The Trump Revolution
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Schumer hints of compromise with Trump, but not on ObamaCare
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump 'Apprentice' alum Leeza Gibbons on the importance of caregiving
---------
---------
[{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'compound': 0.3612}]
---------
---------
Trump ditches press pool
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Behind the scenes of the Trump transition
---------
---------
[{'neg': 0.0, 'neu': 

---------
WikiLeaks reveals damaging Clinton campaign emails
---------
---------
[{'neg': 0.398, 'neu': 0.602, 'pos': 0.0, 'compound': -0.5106}]
---------
---------
Clinton campaign to compare WikiLeaks dump to Watergate
---------
---------
[{'neg': 0.271, 'neu': 0.729, 'pos': 0.0, 'compound': -0.3818}]
---------
---------
Hillary Clinton questioned under oath about email server
---------
---------
[{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.1027}]
---------
---------
Podesta takes swipe at Assange, as his WikiLeaks continues to release hacked Podesta emails
---------
---------
[{'neg': 0.172, 'neu': 0.828, 'pos': 0.0, 'compound': -0.4019}]
---------
---------
Hillary Clinton backers worried about Clinton Foundation scandals
---------
---------
[{'neg': 0.474, 'neu': 0.526, 'pos': 0.0, 'compound': -0.6597}]
---------
---------
WikiLeaks dumps thousands of hacked Clinton campaign emails
---------
---------
[{'neg': 0.474, 'neu': 0.526, 'pos': 0.0, 'compound': -0.6597}]
----

---------
Clinton camp: We felt really strong coming into tonight
---------
---------
[{'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compound': 0.5563}]
---------
---------
Subdued mood at Clinton headquarters
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Many stars devastated over Hillary Clinton loss
---------
---------
[{'neg': 0.558, 'neu': 0.442, 'pos': 0.0, 'compound': -0.743}]
---------
---------
Juan Williams: Clinton legacy deserves better than bitter, angry supporters
---------
---------
[{'neg': 0.341, 'neu': 0.335, 'pos': 0.324, 'compound': -0.0772}]
---------
---------
Gregg Jarrett: Will Trump fulfill his vow to pursue a criminal case against Hillary?
---------
---------
[{'neg': 0.197, 'neu': 0.636, 'pos': 0.168, 'compound': -0.128}]
---------
---------
Greitens: People rejected crooked Clinton-style politics
---------
---------
[{'neg': 0.398, 'neu': 0.602, 'pos': 0.0, 'compound': -0.5106}]
---------
---------
Democrats regrou

TypeError: object of type 'float' has no len()

In [99]:
#most headlines seem to fall under neutral, maybe better to paint directly with the polarity metric

data.head()

,Candidate,Date,Headline,Site,Headline_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Sentiment_Values,Sentiment
0,trump,2016-10-01,Trump and the Intellectuals,New York Times,Trump and the Intellectuals,"Trump, Intellectuals","and, the","Trump, and, the, Intellectuals","[{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'com...",neu
1,trump,2016-10-01,The Other Trump,New York Times,The Other Trump,Trump,"The, Other","The, Other, Trump","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
2,trump,2016-10-01,Trump Infrastructure Plan’s Fatal Flaw,New York Times,Trump Infrastructure Plan’s Fatal Flaw,"Trump, Infrastructure, Plan, Fatal, Flaw",’s,"Trump, Infrastructure, Plan, ’s, Fatal, Flaw","[{'neg': 0.467, 'neu': 0.533, 'pos': 0.0, 'com...",neu
3,trump,2016-10-01,Donald Trump’s Pathetic Fraternity,New York Times,Donald Trump’s Pathetic Fraternity,"Donald, Trump, Pathetic, Fraternity",’s,"Donald, Trump, ’s, Pathetic, Fraternity","[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'com...",neg
4,trump,2016-10-01,How Could Anyone Vote for Trump?,New York Times,How Could Anyone Vote for Trump?,"Vote, Trump, ?","How, Could, Anyone, for","How, Could, Anyone, Vote, for, Trump, ?","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu


In [69]:
#let's try analysing the snippets as well

Doc.set_extension('polarity_scores', getter=polarity_scores,force=True)

data['Snippet_Sentiment_Values'] = ""
data['Snippet_Sentiment'] = ""

for i in data.index.tolist():
    doc = nlp(data['Snippet'][i])
    
    print('---------')
    print(doc)
    print('---------')
                
    senti = []
    senti.append(doc._.polarity_scores)
        
    data.at[i,'Snippet_Sentiment_Values'] = senti   
    data.at[i,'Snippet_Sentiment'] = sorted(senti[0], key=senti[0].__getitem__)[-1]
    
    print('---------')
    print(senti)
    print('---------')

KeyError: 'Snippet'

In [104]:
data.head()

,Candidate,Date,Headline,Site,Headline_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Sentiment_Values,Sentiment,Headline_polarity
0,trump,2016-10-01,Trump and the Intellectuals,New York Times,Trump and the Intellectuals,"Trump, Intellectuals","and, the","Trump, and, the, Intellectuals","[{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'com...",neu,0.3818
1,trump,2016-10-01,The Other Trump,New York Times,The Other Trump,Trump,"The, Other","The, Other, Trump","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0
2,trump,2016-10-01,Trump Infrastructure Plan’s Fatal Flaw,New York Times,Trump Infrastructure Plan’s Fatal Flaw,"Trump, Infrastructure, Plan, Fatal, Flaw",’s,"Trump, Infrastructure, Plan, ’s, Fatal, Flaw","[{'neg': 0.467, 'neu': 0.533, 'pos': 0.0, 'com...",neu,-0.5423
3,trump,2016-10-01,Donald Trump’s Pathetic Fraternity,New York Times,Donald Trump’s Pathetic Fraternity,"Donald, Trump, Pathetic, Fraternity",’s,"Donald, Trump, ’s, Pathetic, Fraternity","[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'com...",neg,-0.5719
4,trump,2016-10-01,How Could Anyone Vote for Trump?,New York Times,How Could Anyone Vote for Trump?,"Vote, Trump, ?","How, Could, Anyone, for","How, Could, Anyone, Vote, for, Trump, ?","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0


In [101]:
data['Sentiment_Values'][2][0]['compound']

-0.5423

In [102]:
data.Headline.iloc[0]

'Trump and the Intellectuals'

In [105]:
data['Headline_polarity'] = data['Headline']

try:
    for i in range(len(data.Sentiment)):
        data['Headline_polarity'].iloc[i] = data['Sentiment_Values'][i][0]['compound']
except IndexError:
    pass

In [74]:
data['Snippet_polarity'] = data['Headline']

for i in range(len(data.Sentiment)):
    data['Snippet_polarity'].iloc[i] = data['Snippet_Sentiment_Values'][i][0]['compound']

IndexError: string index out of range

In [106]:
data.head(30)

,Candidate,Date,Headline,Site,Headline_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Sentiment_Values,Sentiment,Headline_polarity
0,trump,2016-10-01,Trump and the Intellectuals,New York Times,Trump and the Intellectuals,"Trump, Intellectuals","and, the","Trump, and, the, Intellectuals","[{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'com...",neu,0.3818
1,trump,2016-10-01,The Other Trump,New York Times,The Other Trump,Trump,"The, Other","The, Other, Trump","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0
2,trump,2016-10-01,Trump Infrastructure Plan’s Fatal Flaw,New York Times,Trump Infrastructure Plan’s Fatal Flaw,"Trump, Infrastructure, Plan, Fatal, Flaw",’s,"Trump, Infrastructure, Plan, ’s, Fatal, Flaw","[{'neg': 0.467, 'neu': 0.533, 'pos': 0.0, 'com...",neu,-0.5423
3,trump,2016-10-01,Donald Trump’s Pathetic Fraternity,New York Times,Donald Trump’s Pathetic Fraternity,"Donald, Trump, Pathetic, Fraternity",’s,"Donald, Trump, ’s, Pathetic, Fraternity","[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'com...",neg,-0.5719
4,trump,2016-10-01,How Could Anyone Vote for Trump?,New York Times,How Could Anyone Vote for Trump?,"Vote, Trump, ?","How, Could, Anyone, for","How, Could, Anyone, Vote, for, Trump, ?","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0
5,trump,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...,New York Times,Donald Trump Opens New Line of Attack on Hilla...,"Donald, Trump, Opens, New, Line, Attack, Hilla...","of, on, Her","Donald, Trump, Opens, New, Line, of, Attack, o...","[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compo...",neu,-0.4767
6,trump,2016-10-01,Report That Donald Trump Did Business in Cuba ...,New York Times,Report That Donald Trump Did Business in Cuba ...,"Report, Donald, Trump, Business, Cuba, Ups, An...","That, Did, in, the, in","Report, That, Donald, Trump, Did, Business, in...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0
7,trump,2016-10-01,"After Trump-Clinton, Vice-Presidential Debate ...",New York Times,"After Trump-Clinton, Vice-Presidential Debate ...","Trump, -, Clinton, ,, Vice, -, Presidential, D...","After, Is, n’t, the, of","After, Trump, -, Clinton, ,, Vice, -, Presiden...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0
8,trump,2016-10-01,Donald Trump Is Seen as Helping Push Asian-Ame...,New York Times,Donald Trump Is Seen as Helping Push Asian-Ame...,"Donald, Trump, Seen, Helping, Push, Asian, -, ...","Is, as, Into","Donald, Trump, Is, Seen, as, Helping, Push, As...","[{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compo...",neu,0.296
9,trump,2016-10-01,Girl Talk at Trump Tower,New York Times,Girl Talk at Trump Tower,"Girl, Talk, Trump, Tower",at,"Girl, Talk, at, Trump, Tower","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0


In [77]:
#data.drop(columns = ['Snippet_Keywords','Snippet_Stopwords','Snippet_Tokens','Snippet_Sentiment_Values','Snippet_Sentiment'], inplace=True)

# Frequency

In [109]:
#for frquency need to tokenize
#to tokenize need to clear punctuations

from collections import Counter

def wordextractdf(df, col):
    """
    Extract words in the dataframe

    Inputs:
    df -- dataframe
    col -- set of words that we can extract ('keywords', 'hashtags',...)

    Outputs:
    wordsfreq - Counter containing a list of words and its frequency in the dataframe
    """
    words = df[col][df[col].isnull() == False]
    lst = [w for word in words for w in word.split(",")]
    return Counter(lst)

def calc_freq(df, col):
    kw = wordextractdf(df, col)
    dictlist = [[key, value] for key, value in kw.items()]
    freq_df = pd.DataFrame()
    freq_df['words'] = [i[0] for i in dictlist]
    freq_df['freq_total'] = [i[1] for i in dictlist]
    return freq_df

sorted(wordextractdf(data, 'Headline_Keywords').most_common())
freq = calc_freq(data, 'Headline_Keywords')

In [116]:
freq.sort_values(by=['freq_total'], ascending=False)

,words,freq_total
7,Trump,729
29,,477
28,,463
17,Clinton,387
18,:,244
0,Trump,233
6,Donald,187
21,Donald,163
37,’,151
11,?,131


# Export

In [107]:
data.to_csv('data/data_full_sentiment.csv', index=False)